<center> </center>

<center><font size=5 face="Helvetica" color=#EE4B2B><b>
Pytorch Tutorial: Quickstart
</b></font></center>

<center><font face="Helvetica" size=3><b>Ang Chen</b></font></center>
<center><font face="Helvetica" size=3>July, 2024</font></center>

***

# Working with data

Pytorch has two primitives to work with data: $\texttt{torch.utils.data.DataLoader}$ and $\texttt{torch.utils.data.Dataset}$.

$\texttt{Dataset}$ stores the samples and their corresponding labels, and $\texttt{DataLoader}$ wraps an iterable around the $\texttt{Dataset}$.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets. 
For this tutorial, we will be using a TorchVision dataset.

The $\texttt{torchvision.datasets}$ module contains Dataset objects for many real-world vision data like CIFAR, COCO ([full list here](https://pytorch.org/vision/stable/datasets.html)). 
In this tutorial, we use the FashionMNIST dataset. 
Every TorchVision $\texttt{Dataset}$ includes two arguments: $\texttt{transform}$ and $\texttt{target\_transform}$ to modify the samples and labels respectively.

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the $\texttt{Dataset}$ as an argument to the $\texttt{DataLoader}$.
This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess dataloading.
Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [3]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Creating Models

To define a neural network in PyTorch, we create a class that inherits from $\texttt{nn.Module}$.
We define the layers of the network in the function $\texttt{\_\_init\_\_()}$ and specify how data will pass through the network in the $\texttt{forward}$ function.
To accelerate operations in the neural network, we move it to the GPU or MPS (Metal Performance Shaders) if available.

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [5]:
# Define model 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(), 
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
model = NeuralNetwork().to(device=device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Optimizing the Model Parameters

To train a model, we need a loss function and an optimizer.

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model's parameters.

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        

We also check the model’s performance against the test dataset to ensure it is learning.

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

The training process is conducted over several iterations (epochs).
During each epoch, the model learns parameters to make better predictions.
We print the model's accuracy and loss at each epoch; we'd like to see the accuacy increase and loss decrease with every epoch.

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.311330  [    0/60000]
loss: 2.292385  [ 6400/60000]
loss: 2.270203  [12800/60000]
loss: 2.264715  [19200/60000]
loss: 2.242419  [25600/60000]
loss: 2.216673  [32000/60000]
loss: 2.226184  [38400/60000]
loss: 2.188754  [44800/60000]
loss: 2.181112  [51200/60000]
loss: 2.154545  [57600/60000]
Test Error: 
 Accuracy: 40.6%, Avg loss: 2.145596 

Epoch 2
-------------------------------
loss: 2.158462  [    0/60000]
loss: 2.148860  [ 6400/60000]
loss: 2.082153  [12800/60000]
loss: 2.103302  [19200/60000]
loss: 2.055143  [25600/60000]
loss: 1.989991  [32000/60000]
loss: 2.017928  [38400/60000]
loss: 1.931602  [44800/60000]
loss: 1.929319  [51200/60000]
loss: 1.873561  [57600/60000]
Test Error: 
 Accuracy: 57.5%, Avg loss: 1.864825 

Epoch 3
-------------------------------
loss: 1.895431  [    0/60000]
loss: 1.876404  [ 6400/60000]
loss: 1.739406  [12800/60000]
loss: 1.790095  [19200/60000]
loss: 1.694121  [25600/60000]
loss: 1.634913  [32000/600

# Saving Models

The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [13]:
torch.save(model.state_dict(), "./models/model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


The model can now be used to make predictions.

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
